In [1]:
include("./mm.jl");

WebIO._IJuliaInit()

In [2]:
# s is the stock prices. it is the output variable. 
# ϕ is a temporary array which will be used to hold the normal distribution
function weiner(days::Int, μ::Float64, σ::Float64, s::Vector{Float64}, 
        ϕ::Vector{Float64}, dist::Normal{Float64})
    s[1] = 100
    a::Float64 = μ/365.0
    b::Float64 = σ/sqrt(365.0)
    rand!(dist, ϕ)
    for i::Int in 2:days
        s[i] = s[i-1]*(1 + a + ϕ[i]*b)
    end  
    return s
end

function downscale(Y::Vector{Float64}, points::Int)
    days::Int = length(Y)
    step::Float64 = 1
    if days > points
        step = days / points
    end
    println(step)
    XX = Vector{Float64}(undef, 0)
    YY = Vector{Float64}(undef, 0)
    for i in 1:step:days
        push!(XX, Int(i))
        push!(YY, Y[min(days, Int(round(i)))])
    end
    return XX, YY
end
days = 10000
μ, σ = 10.0/100, 30.0/100
# X, Y = downscale(weiner(days, μ, σ), 5000)
# doplot(X, Y)


(0.1, 0.3)

In [ ]:
# Strategy: buy a stock every day, sell it when it hits cagr of 10%. or a maximum of 1000 days
function simulateW()
    runs::Int = 5000
    wallets::Vector{Float64} = Vector{Float64}(undef, runs)
    
    rateThreshold::Float64 = .1; investDays::Int = 365*3; redeemDays::Int=365; days = investDays + redeemDays; 
    μ::Float64=.1; σ::Float64=.3; ll = FastList{Tuple{Int, Float64}}(4000);
    s::Union{Vector{Float64}, Nothing}=Vector{Float64}(undef, days); 
        ϕ::Union{Vector{Float64}, Nothing}=Vector{Float64}(undef, days); dist::Union{Normal{Float64}, Nothing}=Normal(0, 1);
    
    for run::Int in 1:runs
        wallets[run] = oneRun(rateThreshold, investDays, redeemDays, μ, σ, ll, s, ϕ, dist)
        reset!(ll)
    end
    histogram(wallets)
    
end

function oneRun(rateThreshold::Float64, investDays::Int, redeemDays::Int, μ::Float64, σ::Float64, ll::FastList{Tuple{Int, Float64}}, 
        s::Union{Vector{Float64}, Nothing}=nothing, 
        ϕ::Union{Vector{Float64}, Nothing}=nothing, dist::Union{Normal{Float64}, Nothing}=nothing)
    days::Int = investDays+redeemDays
    s = weiner(days, μ, σ, s, ϕ, dist)
#     daysToRedeem = Vector{Int}(undef, days)
#     fill!(daysToRedeem, 0)
#     display(doplot(collect(1:days), s))
    
    minWallet = 0
    wallet::Float64 = 0
    for day::Int in 1:days
        for node in ll
            (d::Int, price::Float64) = node.data
            rate::Float64 = r(price, s[day], (day-d)/365.0)
            if rate > rateThreshold
                wallet += s[day]
                del(ll, node)
#                 daysToRedeem[d] = day-d
            end
        end
        if day <= investDays
            wallet -= s[day]
            push!(ll, (day, s[day]))
        end
        minWallet = min(wallet, minWallet)
    end
    # dispose all the stocks
    wallet += s[end]*ll.len
#     display(plot(fit(Histogram, daysToRedeem, 0:10)))
    return wallet
end
ll = FastList{Tuple{Int, Float64}}(4000)
# oneRun(0.1, 3, 3, .1, .2, ll)
@time simulate()

In [ ]:
# Strategy: invest the same amount every day, sell it when it hits cagr of 10%. or a maximum of 1000 days
function simulateWeiner()
    runs::Int = 5000
    wallets::Vector{Float64} = Vector{Float64}(undef, runs)
    investDays::Int = 365*3; redeemDays::Int=365; days = investDays + redeemDays; 
    ll = FastList{Tuple{Int, Float64, Float64}}(investDays+redeemDays+1000);
    
    
    rateThreshold::Float64 = .1; 
    μ=.1; σ=.3; prices=Vector{Float64}(undef, days); ϕ=Vector{Float64}(undef, days); dist=Normal(0, 1);
    tradingDays = collect(1:days)
    for run::Int in 1:runs
        s = weiner(days, μ, σ, prices, ϕ, dist)
        wallets[run] = oneRun(rateThreshold, investDays, redeemDays,ll, prices)
        reset!(ll)
    end
    #histogram(wallets)
    
end

function oneRun(rateThreshold::Float64, investDays::Int, redeemDays::Int, ll::FastList{Tuple{Int, Float64, Float64}}, 
        s::Union{Vector{Float64}, Nothing}=nothing)
    days::Int = investDays+redeemDays
    amount::Int = 100
    
    minWallet = 0
    wallet::Float64 = 0
    for day::Int in 1:days
        for node in ll
            (d::Int, price::Float64, num::Float64) = node.data
            rate::Float64 = r(price, s[day], (day-d)/365.0)
            if rate > rateThreshold
                wallet += s[day]*num
                del(ll, node)
#                 daysToRedeem[d] = day-d
            end
        end
        if day <= investDays
            wallet -= amount
            push!(ll, (day, s[day], amount/s[day]))
        end
        minWallet = min(wallet, minWallet)
    end
    # dispose all the stocks
    for node in ll
        (d::Int, price::Float64, num::Float64) = node.data
        wallet += s[end]*num
    end
#     display(plot(fit(Histogram, daysToRedeem, 0:10)))
    return wallet
end
ll = FastList{Tuple{Int, Float64}}(4000)
# oneRun(0.1, 3, 3, .1, .2, ll)
@time simulate()